# COGS 108 Final Project

Team Members: Thalal Cassim, Kevin Chen, Yongji Shen, Ansuman Somasundaram, Wei Zeng

# Data Wrangling

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [56]:
# read from data set
suicide = pd.read_csv("./datasets/suicide-rates-overview-1985-to-2016.csv")

In [57]:
#select data from those following countries from our data set
selected = ["Canada","United States","United Kingdom","Australia",'Argentina',"Thailand","Sri Lanka","South Africa"]
tmp = list()   
for i in (suicide.index):
    for country in selected:
        if suicide.loc[i,"country"]==country:
            tmp.append(suicide.loc[i].tolist())
df = pd.DataFrame(columns = suicide.columns)
for i in tmp:
    df = df.append(pd.Series(i, index=suicide.columns), ignore_index=True)

In [58]:
#suicide.isnull().any()
suicideHDI = suicide.dropna()

dfHDI = df.dropna()

In [60]:
#sex,age,dgc,and hdi for all countries
suicide_sex = suicide[['country','year','sex','suicides_no','population']]
suicide_age = suicide[['country','year','age','suicides_no','population']]
suicide_gdp = suicide[['country','year','suicides_no','population',' gdp_for_year ($) ','gdp_per_capita ($)']]
suicide_hdi = suicideHDI[['country','year','suicides_no','population','HDI for year']]

In [64]:
#sex,age,dgc,and hdi for selected countries 
#("Canada","United States","United Kingdom","Australia",'Argentina',"Thailand","Sri Lanka","South Africa")
df_sex = df[['country','year','sex','suicides_no','population']]
df_age = df[['country','year','age','suicides_no','population']]
df_gdp = df[['country','year','suicides_no','population',' gdp_for_year ($) ','gdp_per_capita ($)']]
df_hdi = dfHDI[['country','year','suicides_no','population','HDI for year']]

In [81]:
year = df_sex['year'].unique()
#print(year)
count_male = 0
conut_female = 0
#df_sex
for i in range(len(df_sex)):
    if df_sex.loc[i,"country"] == 'Argentina':
        if df_sex.loc[i,"sex"] == "male":
            count_male += df_sex.loc[i,"suicides_no"]
        else:
            conut_female += df_sex.loc[i,"suicides_no"]
print(count_male)
print(conut_female)
df_sex

63893
18326


,country,year,sex,suicides_no,population
0,Argentina,1985,male,202,363000
1,Argentina,1985,male,485,1997000
2,Argentina,1985,male,414,3346300
...,...,...,...,...,...
2527,United States,2015,female,540,11778666
2528,United States,2015,male,255,21273987
2529,United States,2015,female,158,20342901


# Data Visualization

In [62]:
# Configure Seaborn
sns.set()
sns.set_context('talk')

# Set max number of displayed rows/columns
pd.options.display.max_rows = 7
pd.options.display.max_columns = 8

# Round decimals when displaying DataFrames
pd.set_option('precision', 2)

In [63]:
Fortnite_df['winter.GPA'].plot('hist', bins = 25)
f1 = plt.gcf()

NameError: name 'Fortnite_df' is not defined

In [ ]:
Fortnite_df['cumulative GPA? '].plot('hist', bins = 25)
f2 = plt.gcf()

In [ ]:
Fortnite_df['total.time'].plot('hist', bins = 25)
f3 = plt.gcf()

In [ ]:
pd.plotting.scatter_matrix(Fortnite_df[['winter.GPA', 'cumulative GPA? ', 'total.time']], figsize = (10, 10))
f4 = plt.gcf()

In [ ]:
Sales['Global_Sales'].plot('hist', bins = 25)
f5 = plt.gcf()

In [ ]:
print(Sales['User_Score'])
Sales['User_Score'].plot('hist', bins = 25)
f6 = plt.gcf()

In [ ]:
Sales['User_Count'].plot('hist', bins = 25)
f7 = plt.gcf()

In [ ]:
pd.plotting.scatter_matrix(Sales[['Global_Sales', 'User_Score', 'User_Count']], figsize = (10, 10))
f8 = plt.gcf()

In [ ]:
# Try correlation btwn year/sales, year/scores, sales/scores and vice versa
